In [2]:

import pandas as pd
import vertexai
import os
from dotenv import load_dotenv
import pickle
load_dotenv()

True

In [2]:
data = pd.read_csv("data/movies_metadata.csv")
data.head()

/var/folders/d9/7kl5np_95hz69s_vwscxrc8r0000gn/T/ipykernel_80377/2940964563.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
data.overview[1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."

In [4]:
data.id.isna().sum()

0

In [5]:
data.overview.isna().sum()

954

In [6]:
data.overview = data.overview.fillna("missing")

In [7]:
from utilss import authenticate, encode_text_to_embedding_batched, encode_texts_to_embeddings,generate_batches
# from utilss import authenticate

credentials = authenticate()

In [8]:
vertexai.init(project=os.environ.get('PROJECT_ID'), location=os.environ.get('REGION'),credentials = credentials)

In [9]:
len(data.overview.to_list())

45466

In [10]:
data.overview[:50]

0     Led by Woody, Andy's toys live happily in his ...
1     When siblings Judy and Peter discover an encha...
2     A family wedding reignites the ancient feud be...
3     Cheated on, mistreated and stepped on, the wom...
4     Just when George Banks has recovered from his ...
5     Obsessive master thief, Neil McCauley leads a ...
6     An ugly duckling having undergone a remarkable...
7     A mischievous young boy, Tom Sawyer, witnesses...
8     International action superstar Jean Claude Van...
9     James Bond must unmask the mysterious head of ...
10    Widowed U.S. president Andrew Shepherd, one of...
11    When a lawyer shows up at the vampire's doorst...
12    An outcast half-wolf risks his life to prevent...
13    An all-star cast powers this epic look at Amer...
14    Morgan Adams and her slave, William Shaw, are ...
15    The life of the gambling paradise – Las Vegas ...
16    Rich Mr. Dashwood dies, leaving his second wif...
17    It's Ted the Bellhop's first night on the 

In [11]:
embeddings = encode_text_to_embedding_batched(data.overview.to_list())

# Create a dictionary with movie names as keys and embeddings as values
embedding_dict = dict(zip(data.id.to_list(), embeddings))

# Save the dictionary to a pickle file
with open('movie_embeddings.pickle', 'wb') as f:
    pickle.dump(embedding_dict, f)

number of batches: 9094
estimated time: 909.4000000000001


100%|██████████| 44/44 [01:07<00:00,  1.52s/it]


all batches done


100%|██████████| 45466/45466 [00:00<00:00, 2374703.95it/s]


In [12]:
embeddings.shape

(45341, 768)

In [14]:
# embeddings[0]

In [15]:
# import numpy as np
# np.split(embeddings[0].flatten(),embeddings[0].shape[0])

In [20]:
# Create a dictionary with movie names as keys and embeddings as values
embedding_dict = dict(zip(data.title.to_list(), embeddings))

# Save the dictionary to a pickle file
with open('movie_embeddings.pickle', 'wb') as f:
    pickle.dump(embedding_dict, f)

In [3]:
with open('movie_embeddings.pickle', 'rb') as f:
    embedding_dict = pickle.load(f)

# Example: Accessing embedding for a movie name
movie_name = 'Toy Story'
embedding = embedding_dict.get(movie_name)

if embedding is not None:
    print(f"Embedding for '{movie_name}': {embedding}")
else:
    print(f"No embedding found for '{movie_name}'")

No embedding found for 'Toy Story'


In [5]:
# embedding_dict

In [9]:
for key, value in embedding_dict.items():
    print(f"Embedding for '{key}': {value}")
    break

Embedding for '862': [ 3.02489884e-02 -4.13589254e-02  9.56019200e-03  2.99868193e-02
 -1.80506296e-02  8.71723238e-03  1.71948522e-02 -3.83658498e-03
 -1.32647632e-02  2.86223134e-03  1.99185014e-02 -4.28255610e-02
  1.33261066e-02  3.07962094e-02 -2.42814664e-02  3.12057068e-03
 -1.19505972e-01 -4.12074700e-02  5.40147088e-02 -1.57688130e-02
 -7.96950832e-02 -2.20432524e-02  2.32299864e-02 -3.05052903e-02
 -5.64785488e-03 -9.08620656e-02  9.85122379e-03 -8.26592278e-03
 -1.43104314e-03  2.40977984e-02  1.22294985e-02 -2.88470853e-02
 -2.01734435e-02 -2.32789130e-03 -6.84083672e-03 -1.49856359e-02
 -1.58967022e-02  3.65221873e-02 -5.05637843e-03  1.39299314e-02
 -9.47920023e-04 -4.88123335e-02 -9.57134459e-03 -1.17800236e-02
 -1.96020249e-02 -5.78400074e-03 -4.80071781e-03  7.93387145e-02
  3.59663926e-02 -3.63527685e-02 -3.65163162e-02 -2.84026321e-02
 -2.84647383e-02  7.26782391e-03 -4.39705886e-03 -7.52668548e-03
 -1.53822610e-02 -2.88922898e-03 -2.41690464e-02  2.87479442e-02
  1.